In [1]:
# import warnings
# warnings.filterwarnings("ignore")

# Цель занятия
На этом занятии мы рассмотрим особенности реализаций градиентного бустинга xgboost, lightgbm и catboost.

Мы будем использовать датасет "Bank Marketing". В датасете "Bank Marketing" обычно содержатся данные, связанные с маркетинговыми кампаниями банка. Вот некоторые общие колонки, которые могут присутствовать в датасете "Bank Marketing":

1. **age**: Возраст клиента.

2. **job**: Тип работы клиента.

3. **marital**: Семейное положение клиента.

4. **education**: Образование клиента.

5. **default**: Есть ли у клиента кредитный дефолт (задолженность).

6. **balance**: Баланс на счете клиента.

7. **housing**: Имеет ли клиент ипотеку (жилую недвижимость).

8. **loan**: Имеет ли клиент личный заем.

9. **contact**: Способ связи с клиентом.

10. **day**: День месяца, когда был выполнен последний контакт.

11. **month**: Месяц года, когда был выполнен последний контакт.

12. **duration**: Продолжительность последнего контакта в секундах.

13. **campaign**: Количество контактов, выполненных во время данной кампании.

14. **pdays**: Количество дней, прошедших с момента последнего контакта до предыдущей кампании.

15. **previous**: Количество контактов, выполненных перед текущей кампанией.

16. **poutcome**: Результат предыдущей маркетинговой кампании.

17. **target_variable**: Целевая переменная, которую необходимо предсказать. Например, это может быть информация о том, стал ли клиент подписчиком (1) или нет (0) на определенное предложение банка.

Обратите внимание, что фактические названия колонок и их типы данных могут различаться в зависимости от конкретного датасета "Bank Marketing", поэтому рекомендуется проверить описание колонок в вашем конкретном наборе данных.

In [1]:
# %pip install pandas scikit-learn xgboost lightgbm catboost hyperopt optuna bayesian-optimization

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 2.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 4.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 KB 6.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 6.2 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.9/612.9 KB 5.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Библиотеки общего назначения
import pandas as pd
import time

# Модули метрик
from sklearn.metrics import f1_score

# Модули препроцессинга данных
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Библиотеки градиентного бустинга
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Загрузка данных из CSV файла
url = 'https://datahub.io/machine-learning/bank-marketing/r/bank-marketing.csv'
column_names = ['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 
                'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 
                'poutcome', 'target_variable']

data = pd.read_csv(url)
data.columns = column_names
data.head()

# Разделение на признаки (X) и целевую переменную (y)
X = data.drop('target_variable', axis=1)
y = data['target_variable'].apply(lambda x: x - 1)


# Разделение на обучающий и тестовый наборы данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Определение препроцессора для категориальных и числовых признаков
categorical_features = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
numerical_transformer = StandardScaler()

# Объединение препроцессоров
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

# Создание пайплайна для препроцессора
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Применение препроцессора к данным
X_train = preprocessing_pipeline.fit_transform(X_train)
X_test = preprocessing_pipeline.transform(X_test)

## XGBoost

XGBoost (eXtreme Gradient Boosting) - это библиотека градиентного бустинга, предназначенная для решения задач классификации и регрессии. Она основана на идее градиентного бустинга, который строит ансамбль слабых моделей (обычно деревьев решений) и объединяет их для получения более сильной и устойчивой модели.

Описание алгоритма XGBoost:

1. Инициализация модели:
   - Инициализируется модель с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - Каждое дерево строится поэтапно. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - Для построения дерева используется критерий информативности, такой как критерий Джини или энтропийный критерий, для определения наилучшего разбиения на каждом узле дерева.
   - Деревья строятся с ограничением на их глубину или другими параметрами, чтобы избежать переобучения.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева. Веса определяются скоростью обучения (learning rate), которая контролирует влияние каждого дерева на итоговое предсказание модели.

4. Регуляризация и предотвращение переобучения:
   - Дополнительные механизмы регуляризации в XGBoost помогают предотвратить переобучение и улучшить обобщающую способность модели.
   - XGBoost предлагает несколько методов регуляризации, таких как L1- и L2-регуляризация (также известные как регуляризация Лассо и ридж), которые помогают контролировать сложность модели и предотвращать переобучение. Эти методы добавляют штрафы к функции потерь, которые зависят от весов модели.

6. Функция потерь:
   - XGBoost использует адаптивную функцию потерь, которая сочетает в себе различные функции потерь в зависимости от значения целевой переменной. Например, для задачи классификации с двумя классами может использоваться кросс-энтропия ($L(y, \hat{y}) = -y \log(\hat{y}) - (1-y) \log(1-\hat{y})$), а для задачи регрессии - среднеквадратичная ошибка $(\frac{1}{n}\sum (y - \hat{y})^2)$.
   
7. Ансамблирование деревьев:
   - Поскольку XGBoost строит ансамбль из нескольких деревьев, предсказания каждого дерева складываются, чтобы получить итоговое предсказание модели. Модель объединяет прогнозы всех деревьев с учетом их весов, определенных на основе ошибок и значимости.

8. Прогнозирование:
    - После обучения модели XGBoost можно использовать для прогнозирования на новых данных. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели XGBoost обычно оптимизируют следующие основные параметры:

1. **max_depth**: Глубина дерева. Этот параметр контролирует, насколько глубоким может быть каждое дерево в ансамбле. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр определяет вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **n_estimators**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bytree**: Доля признаков, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **colsample_bylevel**: Доля признаков, используемых для построения каждого дерева на каждом уровне. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

7. **colsample_bynode**: Доля признаков, используемых для построения каждого дерева в каждом узле разбиения. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

8. **gamma**: Минимальное уменьшение функции потерь, необходимое для выполнения расщепления в дереве. Этот параметр контролирует регуляризацию модели. Большее значение gamma помогает предотвратить переобучение за счет увеличения порога для выполнения расщепления.

9. **reg_alpha**: Параметр L1 регуляризации. Он помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

10. **reg_lambda**: Параметр L2 регуляризации. Он помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

In [4]:
# Библиотеки и модули оптимизации
from sklearn.model_selection import GridSearchCV

# Определение модели XGBoost
model = xgb.XGBClassifier()

# Определение сетки параметров для оптимизации
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1],
    'n_estimators': [30]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
grid_search.fit(X_train, y_train)
xgb_gridsearch_execution_time = time.time() - start_time
print("Время работы xgb_gridsearch_execution_time:", xgb_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
xgb_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_gridsearch_f1)

Время работы xgb_gridsearch_execution_time: 49.16833782196045
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 30}
Лучшая F1: 0.4608480661550279
F1 на тестовом наборе данных: 0.4515731030228254


In [5]:
from hyperopt import fmin, tpe, hp, Trials

# Определение функции для оптимизации
def objective(params):
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'n_estimators': hp.choice('n_estimators', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
xgb_hyperopt_execution_time = time.time() - start_time
print("Время работы xgb_hyperopt_execution_time:", xgb_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'max_depth': [3, 4, 5][best['max_depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'n_estimators': [30, 50, 100][best['n_estimators']]
}
# Лучшие параметры и значение метрики
best_score = -trials.best_trial['result']['loss']


print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
xgb_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_hyperopt_f1)

100%|██████████| 10/10 [00:50<00:00,  5.03s/trial, best loss: -0.5251141552511416]
Время работы xgb_hyperopt_execution_time: 50.32681059837341
Лучшие параметры: {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100}
Лучшая F1-мера: 0.5251141552511416
F1 на тестовом наборе данных: 0.5251141552511416


In [6]:
import optuna

# Определение функции для оптимизации
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30, 50, 100])
    }
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
xgb_optuna_execution_time = time.time() - start_time
print("Время работы xgb_optuna_execution_time:", xgb_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = xgb.XGBClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
xgb_optuna_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", xgb_optuna_f1)

[I 2023-06-09 17:01:24,235] A new study created in memory with name: no-name-3d5656c0-6819-4e48-9724-541a37bf00c7
[I 2023-06-09 17:01:28,227] Trial 0 finished with value: 0.43324937027707805 and parameters: {'max_depth': 4, 'learning_rate': 0.019749516996585434, 'n_estimators': 50}. Best is trial 0 with value: 0.43324937027707805.
[I 2023-06-09 17:01:31,699] Trial 1 finished with value: 0.4352791878172589 and parameters: {'max_depth': 4, 'learning_rate': 0.07604569613661794, 'n_estimators': 50}. Best is trial 1 with value: 0.4352791878172589.
[I 2023-06-09 17:01:37,244] Trial 2 finished with value: 0.49095295536791317 and parameters: {'max_depth': 5, 'learning_rate': 0.09607087080427647, 'n_estimators': 50}. Best is trial 2 with value: 0.49095295536791317.
[I 2023-06-09 17:01:44,488] Trial 3 finished with value: 0.5210374639769453 and parameters: {'max_depth': 5, 'learning_rate': 0.09586958138171592, 'n_estimators': 100}. Best is trial 3 with value: 0.5210374639769453.
[I 2023-06-09 17

Время работы xgb_optuna_execution_time: 43.012019872665405
Лучшие параметры: {'max_depth': 5, 'learning_rate': 0.09586958138171592, 'n_estimators': 100}
Лучшая F1-мера: 0.5210374639769453
F1 на тестовом наборе данных: 0.5210374639769453


In [7]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
xgb_bayes_opt_execution_time = time.time() - start_time
print("Время работы xgb_bayes_opt_execution_time:", xgb_bayes_opt_execution_time)

# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели на тестовом наборе данных с лучшими параметрами
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
xgb_bayes_opt_f1 = f1_score(y_test, y_pred)

print("F1-мера на тестовом наборе данных:", xgb_bayes_opt_f1)


|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.4142    | 0.03808   | 4.901     | 81.24     |
| 2         | 0.3472    | 0.06027   | 3.312     | 40.92     |
| 3         | 0.5003    | 0.1       | 4.899     | 79.93     |
| 4         | 0.4778    | 0.07988   | 4.893     | 79.92     |
| 5         | 0.4857    | 0.09815   | 4.853     | 80.03     |
| 6         | 0.4135    | 0.03679   | 4.967     | 79.98     |
| 7         | 0.497     | 0.09629   | 4.755     | 80.02     |
| 8         | 0.4913    | 0.09163   | 4.631     | 80.0      |
| 9         | 0.4852    | 0.08243   | 4.664     | 79.89     |
| 10        | 0.4497    | 0.0634    | 4.712     | 80.17     |
| 11        | 0.4892    | 0.0871    | 4.775     | 79.93     |
| 12        | 0.4109    | 0.03758   | 4.426     | 79.98     |
Время работы xgb_bayes_opt_execution_time: 62.50949716567993
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 4.899137796523717

## LightGBM

Описание алгоритма LightGBM:

1. Инициализация модели:
   - Инициализируется модель с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - LightGBM использует алгоритм градиентного бустинга над решающими деревьями.
   - Деревья строятся поэтапно, аналогично XGBoost. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - LightGBM использует оптимизированную версию алгоритма градиентного бустинга, которая основана на методе обучения по гистограммам. Это позволяет существенно ускорить процесс построения деревьев.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева, аналогично XGBoost.

4. Регуляризация и предотвращение переобучения:
   - LightGBM также предлагает несколько методов регуляризации для предотвращения переобучения модели.
   - Он поддерживает L1-регуляризацию (регуляризацию Лассо) и L2-регуляризацию (регуляризацию ридж), которые добавляют штрафы к функции потерь, аналогично XGBoost.

6. Функция потерь:
   - LightGBM поддерживает различные функции потерь, в зависимости от типа задачи (классификация или регрессия). Для задачи бинарной классификации часто используется логистическая функция потерь, а для задачи регрессии - среднеквадратичная ошибка (MSE), аналогично XGBoost.

7. Ансамблирование деревьев:
   - LightGBM также строит ансамбль из нескольких деревьев, и предсказания каждого дерева складываются для получения итогового предсказания модели.

8. Прогнозирование:
   - После обучения модели LightGBM можно использовать для прогнозирования на новых данных, аналогично XGBoost. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели XGBoost обычно оптимизируют следующие основные параметры:

1. **max_depth**: Глубина дерева. Этот параметр контролирует, насколько глубоким может быть каждое дерево в ансамбле. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр определяет вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **n_estimators**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bytree**: Доля признаков, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку признаков для каждого дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **min_split_gain**: Минимальное уменьшение функции потерь, необходимое для выполнения расщепления в дереве. Этот параметр контролирует регуляризацию модели. Большее значение min_split_gain помогает предотвратить переобучение за счет увеличения порога для выполнения расщепления.

7. **reg_alpha** и **reg_lambda**: Параметры L1 и L2 регуляризации. Они помогают контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score

# Определение модели LightGBM
model = lgb.LGBMClassifier()

# Определение сетки параметров для оптимизации
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.05, 0.01, 0.001],
    'n_estimators': [30, 50, 100]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
grid_search.fit(X_train, y_train)
lgb_gridsearch_execution_time = time.time() - start_time
print("Время работы lgb_gridsearch_execution_time:", lgb_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_gridsearch_f1)


Время работы lgb_gridsearch_execution_time: 52.43806481361389
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Лучшая F1: 0.5387241287197826
F1 на тестовом наборе данных: 0.5427927927927928


In [9]:
from hyperopt import fmin, tpe, hp, Trials

# Определение функции для оптимизации
def objective(params):
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'max_depth': hp.choice('max_depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'n_estimators': hp.choice('n_estimators', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
lgb_hyperopt_execution_time = time.time() - start_time
print("Время работы lgb_hyperopt_execution_time:", lgb_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'max_depth': [3, 4, 5][best['max_depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'n_estimators': [30, 50, 100][best['n_estimators']]
}
best_score = -trials.best_trial['result']['loss']
print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_hyperopt_f1)


100%|██████████| 10/10 [00:02<00:00,  3.44trial/s, best loss: -0.5427927927927928]
Время работы lgb_hyperopt_execution_time: 2.9169037342071533
Лучшие параметры: {'max_depth': 5, 'learning_rate': 0.1, 'n_estimators': 100}
Лучшая F1-мера: 0.5427927927927928
F1 на тестовом наборе данных: 0.5427927927927928


In [10]:
import optuna
import lightgbm as lgb
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_categorical('n_estimators', [30, 50, 100])
    }
    
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
lgb_optuna_execution_time = time.time() - start_time
print("Время работы lgb_optuna_execution_time:", lgb_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = lgb.LGBMClassifier(**best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
lgb_optuna_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_optuna_f1)


[I 2023-06-09 17:04:21,904] A new study created in memory with name: no-name-aca0ddcf-bc75-4cdf-b12e-929e725393c3
[I 2023-06-09 17:04:22,375] Trial 0 finished with value: 0.5020527859237536 and parameters: {'max_depth': 4, 'learning_rate': 0.08601226068626279, 'n_estimators': 100}. Best is trial 0 with value: 0.5020527859237536.
[I 2023-06-09 17:04:22,641] Trial 1 finished with value: 0.4475172847265871 and parameters: {'max_depth': 4, 'learning_rate': 0.07942700863011141, 'n_estimators': 50}. Best is trial 0 with value: 0.5020527859237536.
[I 2023-06-09 17:04:23,165] Trial 2 finished with value: 0.5177304964539008 and parameters: {'max_depth': 5, 'learning_rate': 0.05530366197589997, 'n_estimators': 100}. Best is trial 2 with value: 0.5177304964539008.
[I 2023-06-09 17:04:23,373] Trial 3 finished with value: 0.39004707464694016 and parameters: {'max_depth': 4, 'learning_rate': 0.09043196678597118, 'n_estimators': 30}. Best is trial 2 with value: 0.5177304964539008.
[I 2023-06-09 17:04

Время работы lgb_optuna_execution_time: 3.4038119316101074
Лучшие параметры: {'max_depth': 4, 'learning_rate': 0.0986104733668862, 'n_estimators': 100}
Лучшая F1-мера: 0.5213872832369942
F1 на тестовом наборе данных: 0.5213872832369942


In [11]:
from bayes_opt import BayesianOptimization

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
lgb_bayes_opt_execution_time = time.time() - start_time
print("Время работы lgb_bayes_opt_execution_time:", lgb_bayes_opt_execution_time)

# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели на тестовом наборе данных с лучшими параметрами
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = lgb.LGBMClassifier(**best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
lgb_bayes_opt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", lgb_bayes_opt_f1)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.4035    | 0.03808   | 4.901     | 81.24     |
| 2         | 0.2888    | 0.06027   | 3.312     | 40.92     |
| 3         | 0.5023    | 0.1       | 4.899     | 79.93     |
| 4         | 0.5023    | 0.1       | 4.885     | 79.1      |
| 5         | 0.4663    | 0.1       | 3.0       | 79.26     |
| 6         | 0.2416    | 0.01988   | 4.967     | 75.63     |
| 7         | 0.4357    | 0.06061   | 3.33      | 99.95     |
| 8         | 0.336     | 0.02363   | 4.757     | 96.43     |
| 9         | 0.4181    | 0.08251   | 3.054     | 58.11     |
| 10        | 0.0       | 0.001     | 5.0       | 61.5      |
| 11        | 0.443     | 0.1       | 3.0       | 54.58     |
| 12        | 0.4533    | 0.09301   | 4.966     | 50.72     |
Время работы lgb_bayes_opt_execution_time: 9.43139934539795
Лучшие параметры: {'learning_rate': 0.1, 'max_depth': 4.899134752996972,

## Catboost

Описание алгоритма CatBoost:

1. Инициализация модели:
   - Инициализируется модель CatBoost с нулевыми предсказаниями (например, средним значением целевой переменной для задачи регрессии или логарифмом отношения шансов для задачи классификации).

2. Построение базовых моделей (деревьев решений):
   - CatBoost использует алгоритм градиентного бустинга над решающими деревьями.
   - Деревья строятся поэтапно, аналогично XGBoost и LightGBM. На каждом этапе добавляется новое дерево с учетом остатков, оставшихся после предыдущих деревьев.
   - CatBoost применяет особый подход к кодированию категориальных признаков, называемый симметричным бинарным кодированием, который учитывает взаимодействия между категориями и признаками.

3. Вычисление градиентов и обновление предсказаний:
   - После построения каждого дерева вычисляются градиенты ошибки между предсказаниями модели и истинными значениями целевой переменной.
   - Предсказания модели обновляются путем добавления взвешенной версии предсказаний нового дерева, аналогично XGBoost и LightGBM.

4. Регуляризация и предотвращение переобучения:
   - CatBoost также предлагает несколько методов регуляризации для предотвращения переобучения модели.
   - Он поддерживает L1-регуляризацию (регуляризацию Лассо) и L2-регуляризацию (регуляризацию ридж), которые добавляют штрафы к функции потерь, аналогично XGBoost и LightGBM.

6. Функция потерь:
   - CatBoost поддерживает различные функции потерь, в зависимости от типа задачи (классификация или регрессия). Для задачи бинарной классификации часто используется логистическая функция потерь, а для задачи регрессии - среднеквадратичная ошибка (MSE), аналогично XGBoost и LightGBM.

7. Ансамблирование деревьев:
   - CatBoost также строит ансамбль из нескольких деревьев, и предсказания каждого дерева складываются, чтобы получить итоговое предсказание модели.
   - CatBoost также учитывает веса деревьев на основе их ошибок и значимости, аналогично XGBoost и LightGBM.

8. Прогнозирование:
   - После обучения модели CatBoost можно использовать для прогнозирования на новых данных. Модель принимает входные признаки и возвращает прогнозы для задач классификации или регрессии.

При оптимизации модели CatBoost можно обратить внимание на следующие важные параметры:

1. **depth**: Глубина дерева. Этот параметр определяет, насколько глубоким может быть каждое дерево. Большая глубина может привести к переобучению, поэтому его следует настраивать осторожно.

2. **learning_rate**: Скорость обучения. Этот параметр контролирует вклад каждого дерева в ансамбле. Более низкое значение learning_rate требует большего числа деревьев для достижения хорошей производительности, но может улучшить обобщающую способность модели.

3. **iterations**: Количество деревьев в ансамбле. Этот параметр указывает, сколько деревьев следует построить. Большее количество деревьев может улучшить производительность модели, но слишком большое число может привести к переобучению.

4. **subsample**: Доля обучающих примеров, используемых для построения каждого дерева. Этот параметр контролирует случайную подвыборку данных для каждого дерева. Значение меньше 1.0 позволяет уменьшить переобучение и повысить устойчивость модели.

5. **colsample_bylevel**: Доля признаков, используемых для построения каждого уровня дерева. Этот параметр контролирует случайную подвыборку признаков для каждого уровня дерева. Значение меньше 1.0 помогает уменьшить переобучение и повысить разнообразие деревьев.

6. **l2_leaf_reg**: L2 регуляризация. Этот параметр помогает контролировать сложность модели и предотвращать переобучение путем добавления штрафов за большие значения весов.

7. **random_strength**: Сила случайности для ускорения обучения. Этот параметр контролирует уровень случайности в выборе признаков при построении дерева. Значение больше 0 добавляет случайность, что может помочь предотвратить переобучение.

8. **bagging_temperature**: Температура бэггинга. Этот параметр контролирует степень случайности в выборе объектов для обучения каждого дерева. Значение больше 0 добавляет случайность, что может помочь предотвратить переобучение.

Важно отметить, что конкретные параметры и их значения могут варьироваться в зависимости от конкретной задачи и набора данных. Рекомендуется провести эксперименты и настраивать параметры с использованием кросс-валидации для достижения наилучшей производительности модели.

In [12]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

# Определение модели CatBoost
model = CatBoostClassifier(silent=True)

# Определение сетки параметров для оптимизации
# param_grid = {
#     'depth': [3, 4, 5],
#     'learning_rate': [0.1, 0.05, 0.01],
#     'n_estimators': [30, 50, 100]
# }
param_grid = {
    'depth': [5],
    'learning_rate': [0.1, 0.05, 0.01],
    'n_estimators': [30, 50, 100]
}

# Создание объекта GridSearchCV для оптимизации параметров
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')

# Обучение модели с использованием GridSearchCV
start_time = time.time()
grid_search.fit(X_train, y_train)
catboost_gridsearch_execution_time = time.time() - start_time
print("Время работы catboost_gridsearch_execution_time:", catboost_gridsearch_execution_time)

# Лучшие параметры и значение метрики
print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая F1:", grid_search.best_score_)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **grid_search.best_params_)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
catboost_gridsearch_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", catboost_gridsearch_f1)


Время работы catboost_gridsearch_execution_time: 56.67308831214905
Лучшие параметры: {'depth': 5, 'learning_rate': 0.1, 'n_estimators': 100}
Лучшая F1: 0.5182567322214451
F1 на тестовом наборе данных: 0.5148514851485149


In [13]:
from hyperopt import fmin, tpe, hp, Trials
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(params):
    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return -f1_score(y_test, y_pred)  # Целевая метрика, отрицательная F1-мера

# Определение пространства поиска параметров
space = {
    'depth': hp.choice('depth', [3, 4, 5]),
    'learning_rate': hp.choice('learning_rate', [0.1, 0.01, 0.001]),
    'iterations': hp.choice('iterations', [30, 50, 100])
}

# Выполнение оптимизации
trials = Trials()
start_time = time.time()
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)
catboost_hyperopt_execution_time = time.time() - start_time
print("Время работы catboost_hyperopt_execution_time:", catboost_hyperopt_execution_time)

# Лучшие параметры и значение метрики
best_params = {
    'depth': [3, 4, 5][best['depth']],
    'learning_rate': [0.1, 0.01, 0.001][best['learning_rate']],
    'iterations': [30, 50, 100][best['iterations']]
}
best_score = -trials.best_trial['result']['loss']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **best_params)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
catboost_hyperopt_f1 = f1_score(y_pred, y_test)
print("F1 на тестовом наборе данных:", catboost_hyperopt_f1)


100%|██████████| 10/10 [00:14<00:00,  1.46s/trial, best loss: -0.5148514851485149]
Время работы catboost_hyperopt_execution_time: 14.63330340385437
Лучшие параметры: {'depth': 5, 'learning_rate': 0.1, 'iterations': 100}
Лучшая F1-мера: 0.5148514851485149
F1 на тестовом наборе данных: 0.5148514851485149


In [14]:
import optuna

# Определение функции для оптимизации
def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 3, 5),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'iterations': trial.suggest_categorical('iterations', [30, 50, 100])
    }
    
    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)
    
    return f1  # Целевая метрика, максимизируем F1-меру

# Создание объекта Study для оптимизации
study = optuna.create_study(direction='maximize')
start_time = time.time()
study.optimize(objective, n_trials=10)
catboost_optuna_execution_time = time.time() - start_time
print("Время работы catboost_hyperopt_execution_time:", catboost_optuna_execution_time)

# Лучшие параметры и значение метрики
best_params = study.best_params
best_score = study.best_value

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_score)

# Оценка качества модели на тестовом наборе данных
model = CatBoostClassifier(silent=True, **best_params)
model.fit(X_train, y_train)
catboost_optuna_f1 = f1_score(y_test, model.predict(X_test))
print("F1 на тестовом наборе данных:", catboost_optuna_f1)


[I 2023-06-09 17:05:51,364] A new study created in memory with name: no-name-199fe588-80c0-4e6e-b528-9272cf6d0535
[I 2023-06-09 17:05:53,344] Trial 0 finished with value: 0.2867981790591806 and parameters: {'depth': 3, 'learning_rate': 0.013007437402729593, 'iterations': 100}. Best is trial 0 with value: 0.2867981790591806.
[I 2023-06-09 17:05:54,288] Trial 1 finished with value: 0.2911010558069381 and parameters: {'depth': 4, 'learning_rate': 0.003132738396769772, 'iterations': 30}. Best is trial 1 with value: 0.2911010558069381.
[I 2023-06-09 17:05:55,302] Trial 2 finished with value: 0.4251458198314971 and parameters: {'depth': 5, 'learning_rate': 0.08643802891965559, 'iterations': 30}. Best is trial 2 with value: 0.4251458198314971.
[I 2023-06-09 17:05:56,373] Trial 3 finished with value: 0.3671232876712328 and parameters: {'depth': 3, 'learning_rate': 0.06498590597136844, 'iterations': 50}. Best is trial 2 with value: 0.4251458198314971.
[I 2023-06-09 17:05:57,314] Trial 4 finishe

Время работы catboost_hyperopt_execution_time: 12.2300283908844
Лучшие параметры: {'depth': 4, 'learning_rate': 0.09344627089016184, 'iterations': 100}
Лучшая F1-мера: 0.5029655990510082
F1 на тестовом наборе данных: 0.5029655990510082


In [15]:
from catboost import CatBoostClassifier
from bayes_opt import BayesianOptimization
from sklearn.metrics import f1_score

# Определение функции для оптимизации
def objective(max_depth, learning_rate, n_estimators):
    params = {
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators)
    }
    model = CatBoostClassifier(silent=True, **params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return f1

# Определение диапазона значений параметров для оптимизации
pbounds = {
    'max_depth': (3, 5),
    'learning_rate': (0.001, 0.1),
    'n_estimators': (30, 100)
}

# Создание объекта BayesianOptimization для оптимизации
optimizer = BayesianOptimization(
    f=objective,
    pbounds=pbounds,
    random_state=42,
    verbose=2
)

# Запуск оптимизации
start_time = time.time()
optimizer.maximize(init_points=2, n_iter=10)
catboost_bayes_opt_execution_time = time.time() - start_time
print("Время работы catboost_bayes_opt_execution_time:", catboost_bayes_opt_execution_time)


# Лучшие параметры и значение метрики
best_params = optimizer.max['params']
best_f1 = optimizer.max['target']

print("Лучшие параметры:", best_params)
print("Лучшая F1-мера:", best_f1)

# Оценка качества модели с лучшими параметрами на тестовом наборе данных
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['max_depth'] = int(best_params['max_depth'])
best_model = CatBoostClassifier(silent=True, **best_params)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
catboost_bayes_opt_f1 = f1_score(y_test, y_pred)
print("F1 на тестовом наборе данных:", catboost_bayes_opt_f1)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | 0.3813    | 0.03808   | 4.901     | 81.24     |
| 2         | 0.3152    | 0.06027   | 3.312     | 40.92     |
| 3         | 0.4875    | 0.1       | 4.899     | 79.93     |
| 4         | 0.4628    | 0.07988   | 4.893     | 79.92     |
| 5         | 0.4901    | 0.09815   | 4.853     | 80.03     |
| 6         | 0.4138    | 0.04491   | 4.943     | 80.25     |
| 7         | 0.4458    | 0.06957   | 4.62      | 79.91     |
| 8         | 0.3925    | 0.03552   | 4.936     | 80.06     |
| 9         | 0.383     | 0.03166   | 4.748     | 79.98     |
| 10        | 0.4613    | 0.07458   | 4.562     | 79.84     |
| 11        | 0.3579    | 0.02203   | 4.473     | 79.87     |
| 12        | 0.4371    | 0.06125   | 4.562     | 79.83     |
Время работы catboost_bayes_opt_execution_time: 19.0199613571167
Лучшие параметры: {'learning_rate': 0.09815464422786867, 'max_depth

In [16]:
# Создание данных для таблицы сравнения по f1_score и времени работы
params_match = [
    ['XGBoost gridsearch', f'{xgb_gridsearch_f1}', f'{xgb_gridsearch_execution_time}'],
    ['XGBoost hyperopt', f'{xgb_hyperopt_f1}', f'{xgb_hyperopt_execution_time}'],
    ['XGBoost optuna', f'{xgb_optuna_f1}', f'{xgb_optuna_execution_time}'],
    ['XGBoost bayes_opt', f'{xgb_bayes_opt_f1}', f'{xgb_bayes_opt_execution_time}'],
    ['LightGBM gridsearch', f'{lgb_gridsearch_f1}', f'{lgb_gridsearch_execution_time}'],
    ['LightGBM hyperopt', f'{lgb_hyperopt_f1}', f'{lgb_hyperopt_execution_time}'],
    ['LightGBM optuna', f'{lgb_optuna_f1}', f'{lgb_optuna_execution_time}'],
    ['LightGBM bayes_opt', f'{lgb_bayes_opt_f1}', f'{lgb_bayes_opt_execution_time}'],
    ['Catboost gridsearch', f'{catboost_gridsearch_f1}', f'{catboost_gridsearch_execution_time}'],
    ['Catboost hyperopt', f'{catboost_hyperopt_f1}', f'{catboost_hyperopt_execution_time}'],
    ['Catboost optuna', f'{catboost_optuna_f1}', f'{catboost_optuna_execution_time}'],
    ['Catboost bayes_opt', f'{catboost_bayes_opt_f1}', f'{catboost_bayes_opt_execution_time}'],

]

# Создание DataFrame из данных
df = pd.DataFrame(params_match, columns=['Модель и оптимайзер', 'F1', 'Время'])
pd.set_option('display.max_colwidth', None)
# Отображение DataFrame
df.head(len(params_match))

,Модель и оптимайзер,F1,Время
0,XGBoost gridsearch,0.4515731030228254,49.16833782196045
1,XGBoost hyperopt,0.5251141552511416,50.32681059837341
2,XGBoost optuna,0.5210374639769453,43.012019872665405
3,XGBoost bayes_opt,0.5002935995302408,62.50949716567993
4,LightGBM gridsearch,0.5427927927927928,52.43806481361389
5,LightGBM hyperopt,0.5427927927927928,2.9169037342071533
6,LightGBM optuna,0.5213872832369942,3.4038119316101074
7,LightGBM bayes_opt,0.5023474178403756,9.43139934539795
8,Catboost gridsearch,0.5148514851485149,56.67308831214905
9,Catboost hyperopt,0.5148514851485149,14.63330340385437


In [3]:
# Создание данных для таблицы
params_match = [
    ['Скорость обучения (Learning Rate)', 'eta (learning_rate)', 'learning_rate', 'learning_rate'],
    ['Количество деревьев (Number of Trees)', 'n_estimators', 'n_estimators', 'iterations'],
    ['Максимальная глубина дерева (Maximum Depth)', 'max_depth', 'max_depth', 'max_depth'],
    ['Минимальный вес листа дерева, Минимальное количество объектов в листе (Minimum Child Weight)', 'min_child_weight', 'min_child_samples', 'min_data_in_leaf'],
    ['Гамма (Gamma)', 'gamma', 'min_split_gain', 'l2_leaf_reg'],
    ['Доля выборки объектов (Subsample Ratio)', 'subsample', 'subsample', 'subsample'],
    ['Доля выборки признаков (Column Subsampling Ratio)', 'colsample_bytree', 'colsample_bytree', 'colsample_bylevel'],
    ['Лямбда (L2-регуляризация)', 'reg_lambda', 'reg_lambda', 'reg_lambda'],
    ['Альфа (L1-регуляризация)', 'reg_alpha', 'reg_alpha', 'reg_alpha'],
    ['Максимальное изменение шага (Maximum Delta Step)', 'max_delta_step', '-', '-'],
    ['Максимальное количество корзин (Maximum Bin Count)', 'max_bin', 'max_bin', 'max_bin'],
    ['Частота сэмплирования объектов (Sampling Frequency for LGBM)', '-', 'subsample_freq', '-'],
    ['Масштабирование веса положительного класса (Scale the Weight of Positive Class)', 'scale_pos_weight', 'scale_pos_weight', 'scale_pos_weight'],
    ['Режим обучения с помощью градиентного спуска (Booster Type)', 'booster', 'boosting_type', 'boosting_type'],
    ['Коэффициент сжатия (Subsample)', 'subsample', 'subsample', 'subsample'],
    ['Размер выборки для построения дерева (Bagging Fraction)', 'colsample_bytree', 'colsample_bytree', 'colsample_bylevel'],
    ['Способ выбора ветвления (Split Criterion)', 'tree_method', 'tree_learner', 'grow_policy'],
    ['Функция потерь (Loss Function)', 'objective', 'objective', 'loss_function'],
    ['Максимальное количество листьев (Max Leaf Nodes)', '-', 'num_leaves', 'max_leaves']
]

# Создание DataFrame из данных
df = pd.DataFrame(params_match, columns=['Описание параметра', 'XGBoost', 'LightGBM', 'CatBoost'])
pd.set_option('display.max_colwidth', None)
# Отображение DataFrame
df


,Описание параметра,XGBoost,LightGBM,CatBoost
0,Скорость обучения (Learning Rate),eta (learning_rate),learning_rate,learning_rate
1,Количество деревьев (Number of Trees),n_estimators,n_estimators,iterations
2,Максимальная глубина дерева (Maximum Depth),max_depth,max_depth,max_depth
3,"Минимальный вес листа дерева, Минимальное количество объектов в листе (Minimum Child Weight)",min_child_weight,min_child_samples,min_data_in_leaf
4,Гамма (Gamma),gamma,min_split_gain,l2_leaf_reg
5,Доля выборки объектов (Subsample Ratio),subsample,subsample,subsample
6,Доля выборки признаков (Column Subsampling Ratio),colsample_bytree,colsample_bytree,colsample_bylevel
7,Лямбда (L2-регуляризация),reg_lambda,reg_lambda,reg_lambda
8,Альфа (L1-регуляризация),reg_alpha,reg_alpha,reg_alpha
9,Максимальное изменение шага (Maximum Delta Step),max_delta_step,-,-
